In [324]:
import pandas as pd
import numpy as np

In [325]:
data = pd.read_excel("tumor-data-without-healthy.xlsx")

In [326]:
labels = [value for column, value in data.iloc[0].items()]
data = data.set_axis(labels, axis=1)
data = data[data.columns[1:]].apply(
    lambda x: pd.to_numeric(x.replace('<', '').replace(',', '.'), errors='coerce')
)
data.drop(index=0, inplace=True)

In [327]:
data["Гормональная активность 0-нет               1-да"] = data[
    "Гормональная активность 0-нет               1-да"].astype(float)
data.rename(columns={"Гормональная активность 0-нет               1-да":"hormones_activity"}, inplace=True)

In [328]:
data.columns = data.columns.astype(str)
data.columns = [col.replace(' ', '_').replace('-', '_').replace(':', '').replace('(', '').replace(')', '').replace(',', '') for col in data.columns]

In [329]:
""" dropping some columns """
data.dropna(axis=1, how="all", inplace=True)
# data.dropna(axis=0, thresh=int(data.shape[1] * 0.6), inplace=True)
data.dropna(axis=1, thresh=int(data.shape[0] * 0.5), inplace=True)
data.drop(["Пол____0_жен_1_муж"], inplace=True, axis=1)

In [330]:
data

,КАН,АКР,hormones_activity,Максимальный_размер_мм,Нативная_плотность_НU_,APW_%_КВ____коэф_вымывания,Weiss,Кортизол_крови_утро,Кортизол_крови_вечер,Кортизол_после_пробы_с_1_мг_дексаметазона_ДМТ,...,THE_P3,THB_THA,THF_THE1,THF_THE,aTHF_THF,aTHB_THB,11An_11Et,THE_THS,THE_11oxoP3,16_OHdP2_3a/16_OHdP2_3b
1,0.0,1.0,1.0,181.0,NaN,NaN,7.0,NaN,NaN,NaN,...,0.722438,0.268293,0.837273,0.276251,2.030846,26.136364,2.827039,0.312336,334.200000,1.205818
2,0.0,1.0,1.0,137.0,NaN,NaN,6.0,681.0,739.0,NaN,...,17.868078,5.948276,1.601304,1.245525,0.285646,0.289855,0.670094,8.016807,353.903226,0.795526
3,0.0,1.0,1.0,79.0,48.0,NaN,4.0,1418.0,1021.0,NaN,...,3.100000,1.000000,0.950000,0.770000,0.230000,0.170000,1.780000,16.680000,25.010000,2.530000
4,0.0,1.0,1.0,54.0,18.0,53.330000,3.0,687.0,628.0,NaN,...,5.308889,10.000000,0.863495,0.692278,0.247324,2.400000,5.544041,27.941520,227.523810,2.403478
5,0.0,1.0,1.0,NaN,NaN,64.583333,NaN,NaN,NaN,NaN,...,0.558894,NaN,0.964336,0.883217,0.091845,0.465753,1.983240,1.279727,85.121212,1.422500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,1.0,0.0,0.0,57.0,16.0,48.570000,2.0,304.0,113.0,39.0,...,4.273333,0.959184,0.413451,0.198456,1.083333,1.872340,2.837771,36.214689,NaN,1.371429
161,1.0,0.0,1.0,36.0,NaN,96.000000,2.0,344.0,328.0,344.0,...,1.991166,3.255814,0.648263,0.365631,0.773000,2.371429,1.818519,5.053812,13.828221,NaN
162,1.0,0.0,1.0,17.0,8.0,96.428571,0.0,784.0,245.0,NaN,...,37.064646,1.168342,0.929232,0.514090,0.807527,1.247312,3.576531,NaN,1528.916667,NaN
163,1.0,0.0,1.0,43.0,32.0,50.000000,9.0,628.0,345.0,60.4,...,3.629134,4.057143,0.628047,0.307312,1.043678,1.577465,5.618234,219.476190,307.266667,3.089655


In [331]:
X = data.drop(columns=["КАН", "АКР"])
y = np.argmax(data[["КАН", "АКР"]], axis=1)

In [332]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=63)

In [333]:
from lightgbm import LGBMClassifier
import optuna

def objective(trial):
    param = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }

    model = LGBMClassifier(**param, n_jobs=-1, random_state=42)
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2024-12-24 16:20:56,296] A new study created in memory with name: no-name-5826bb64-204a-4a6f-bc54-be49f3b2900b
[I 2024-12-24 16:20:56,315] Trial 0 finished with value: 0.29411764705882354 and parameters: {'lambda_l1': 0.004305492701900274, 'lambda_l2': 0.09844153800492698, 'num_leaves': 201, 'feature_fraction': 0.5543664598383194, 'bagging_fraction': 0.5239395909244918, 'bagging_freq': 4, 'min_child_samples': 63}. Best is trial 0 with value: 0.29411764705882354.
[I 2024-12-24 16:20:56,335] Trial 1 finished with value: 1.0 and parameters: {'lambda_l1': 0.00025034952938701906, 'lambda_l2': 0.00032194494097473813, 'num_leaves': 147, 'feature_fraction': 0.8220024693126398, 'bagging_fraction': 0.8942687335531547, 'bagging_freq': 5, 'min_child_samples': 20}. Best is trial 1 with value: 1.0.
[I 2024-12-24 16:20:56,345] Trial 2 finished with value: 0.29411764705882354 and parameters: {'lambda_l1': 5.4070302087980486e-06, 'lambda_l2': 0.019971125129378257, 'num_leaves': 201, 'feature_fractio

In [334]:
study.best_params, study.best_value

({'lambda_l1': 0.00025034952938701906,
  'lambda_l2': 0.00032194494097473813,
  'num_leaves': 147,
  'feature_fraction': 0.8220024693126398,
  'bagging_fraction': 0.8942687335531547,
  'bagging_freq': 5,
  'min_child_samples': 20},
 1.0)

In [335]:
model = LGBMClassifier(**study.best_params, random_state=63)

In [338]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=314)

cv = cross_validate(model, X, y, n_jobs=-1,
                    scoring=["accuracy",
                             "f1",
                             "precision",
                             "recall"],
                    cv=skf,
                    return_estimator=True)

 accuracy 1.0 
 f1 1.0 
 recall 1.0 
 precision 1.0 

---------------------------------------------
 accuracy 0.9411764705882353 
 f1 0.9411764705882353 
 recall 1.0 
 precision 0.8888888888888888 

---------------------------------------------
 accuracy 0.8823529411764706 
 f1 0.875 
 recall 0.875 
 precision 0.875 

---------------------------------------------
 accuracy 0.8823529411764706 
 f1 0.8888888888888888 
 recall 1.0 
 precision 0.8 

---------------------------------------------
 accuracy 1.0 
 f1 1.0 
 recall 1.0 
 precision 1.0 

---------------------------------------------
 accuracy 0.9375 
 f1 0.9333333333333333 
 recall 0.875 
 precision 1.0 

---------------------------------------------


In [339]:
cv["test_accuracy"].mean()

0.9518382352941177

In [ ]:
for i in range(len(cv) - 1):
    print(
        " accuracy", cv["test_accuracy"][i], "\n",
        "f1", cv["test_f1"][i], "\n",
        "recall", cv["test_recall"][i], "\n",
        "precision", cv["test_precision"][i], "\n"
    )
    print("---------------------------------------------")